In [16]:
%pip install crewai litellm

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install duckduckgo-search
%pip install crewai
%pip install libsqlite3-dev -update
%pip install pysqlite3-binary


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -u
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 38.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [11]:
%python3 -m pip install --upgrade pip

UsageError: Line magic function `%python3` not found (But cell magic `%%python3` exists, did you mean that instead?).


In [12]:
%pip install --upgrade pysqlite3-binary

Note: you may need to restart the kernel to use updated packages.


In [17]:
import sqlite3
print(sqlite3.sqlite_version)

3.31.1


In [15]:
import os
from crewai import Agent, Task, Crew, Process
from litellm import completion
import pandas as pd
import requests
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

RuntimeError: [91mYour system has an unsupported version of sqlite3. Chroma                     requires sqlite3 >= 3.35.0.[0m
[94mPlease visit                     https://docs.trychroma.com/troubleshooting#sqlite to learn how                     to upgrade.[0m

In [28]:
search = DuckDuckGoSearchRun()

In [7]:
# Set Azure OpenAI credentials
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_KEY"] = "secNbxX4Pg4wp8rscewBm1kSis1tFimU"
os.environ["OPENAI_API_BASE"] = "https://Phi-3-5-mini-instruct-iplvi.eastus2.models.ai.azure.com"
os.environ["OPENAI_API_VERSION"] = "2024-02-15-preview"  # Update this to your Azure API version

In [ ]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

In [11]:
# Configure LiteLLM for Azure
completion_model = {
    "model": "Phi-3-5-mini-instruct-iplvi",
    "api_type": "azure",
    "api_key": os.getenv("AZURE_API_KEY"),
    "api_base": os.getenv("AZURE_API_BASE"),
    "api_version": "2024-02-15-preview"  # Update this to your Azure API version
}

In [19]:
# Create the planner agent
Researcher = Agent(
    role="Expert Researcher",
    goal="Research factually accurate content about leadership of {company}, a fortune 500 company",
    backstory="""You're are a journalist for a reputed American publisher
              You have been tracking {company} and it's leadership for many years
              You collect information that helps board members make informed decisions
              about choice of leadership hire and promotion.""",
    allow_delegation=False,
    llm_config=completion_model,  # Use llm_config instead of llm
    verbose=True
)

In [22]:
Stock_Analyst = Agent(
    role="FINRA stock analyst",
    goal="Evaluate health of the company post an event",
    backstory="Your are a SRO certified FINRA stock analyst"
              "You have been tracking {company} and it's leadership changes for many years"
              "Have a master's degree in finance or Chartered Financial Analyst (CFA) designation",
    allow_delegation=False,
    llm_config=completion_model,
    verbose=True
)

In [23]:
Psychologist = Agent(
    role="Industrial-Organizational (I-O) Psychologist",
    goal="Assess the personality type of the leader",
    backstory="You are a PhD holder in Industrial psychology"
              "You specialize in evaluating the personality types of executives"
              "your PhD thesies was evaluating personality type based on what people say and write"
              "You Have expertise in understanding personality types"
              "You deeply Understand how personality traits relate to job performance"
              "You Can interpret complex psychological data in a business context",
    allow_delegation=False,
    llm_config=completion_model,
    verbose=True
)

In [33]:
# Create a task
Find_CEO= Task(
    description="Find out who is or was the CEO of the fortune 500 company {company}"
    "Findout more about the CEO, their education background, pervious roles associated with their education",
    agent=Researcher,
    expected_output="Full name of the CEO"
    "CEO's Background",
    search_tool=search,
    verbose=True
)

In [34]:
Find_CEO_Crew = Crew(
  agents=[
    Researcher
  ],
  tasks=[
    Find_CEO
  ],
  verbose=True
)

In [35]:
inputs = {
  'company': 'Walmart'
}

Result = Find_CEO_Crew.kickoff(
  inputs=inputs
)

# Agent: Expert Researchers
## Task: Find out who is or was the CEO of the fortune 500 company WalmartFindout more about the CEO, their education background, pervious roles associated with their education


# Agent: Expert Researchers
## Final Answer: 
Full Name of the CEO: Doug McMillon

CEO's Background:

Doug McMillon, born on March 21, 1969, is the current CEO of Walmart, Inc. McMillon has an extensive background in retail, having started his career at Sam's Club in Dallas, Texas. He has held various positions at Walmart throughout his career, including the roles of Vice President of Walmart Stores, Executive Vice President and Chief Operating Officer, and eventually, President and CEO.

Education:

Doug McMillon completed his Bachelor's degree in Business Administration at the University of Texas at Dallas. Prior to his retail career, McMillon also attended the U.S. Army's Officer Candidate School, where he earned his Associate in Arts degree in American Studies.

Previous Roles:


In [40]:
Result

CrewOutput(raw="Full Name of the CEO: Doug McMillon\n\nCEO's Background:\n\nDoug McMillon, born on March 21, 1969, is the current CEO of Walmart, Inc. McMillon has an extensive background in retail, having started his career at Sam's Club in Dallas, Texas. He has held various positions at Walmart throughout his career, including the roles of Vice President of Walmart Stores, Executive Vice President and Chief Operating Officer, and eventually, President and CEO.\n\nEducation:\n\nDoug McMillon completed his Bachelor's degree in Business Administration at the University of Texas at Dallas. Prior to his retail career, McMillon also attended the U.S. Army's Officer Candidate School, where he earned his Associate in Arts degree in American Studies.\n\nPrevious Roles:\n\n- Walmart Stores Vice President: McMillon worked his way up from an entry-level position at Sam's Club, gaining valuable experience in different aspects of the retail industry.\n- Executive Vice President and COO: Here, McMi

In [8]:
url = "https://fortune.com/ranking/global500/"
tables = pd.read_html(url)

In [12]:
df_f5_2024 = tables[0]

In [16]:
url = "https://fortune.com/ranking/global500/2004/"
tables = pd.read_html(url)
df_f5_2004 = tables[0]

,RemoveRank,RemoveName,RemoveRevenues ($M),RemoveRevenue Percent Change,RemoveProfits ($M),RemoveProfits Percent Change,RemoveAssets ($M),RemoveEmployees,RemoveChange in Rank,RemoveYears on Global 500 List,Unnamed: 10
0,1,Walmart,"$648,125",6%,"$15,511",32.8%,"$252,399",2100000,-,30,View More details about Walmart
1,2,Amazon,"$574,785",11.8%,"$30,425",-,"$527,854",1525000,2,16,View More details about Amazon
2,3,State Grid,"$545,947.5",3%,"$9,204.3",12.4%,"$781,126.2",1361423,-,24,View More details about State Grid
3,4,Saudi Aramco,"$494,890.1",-18%,"$120,699.3",-24.1%,"$660,819.2",73311,-2,6,View More details about Saudi Aramco
4,5,Sinopec Group,"$429,699.7",-8.8%,"$9,393.4",-2.7%,"$382,688",513434,1,26,View More details about Sinopec Group
